# Lab1 - crypto - Lukas Forst
forstluk@fel.cvut.cz

## Exercise 0: make utilities

This exercise is not strictly mandatory, but it will be useful for the
rest of the lab.

Write 6 functions `bin2txt`, `bin2hex`, `txt2bin`, `hex2bin`, `hex2txt`, `txt2hex`, that convert between the following representations:
  - hex: `"426f6f6d"` (more precisely, a string to be interepreted as
      hexadecimal)
  - text: `"Boom"`
  - binary: `b"Boom"` in Python

Depending on the language, you may not have to distinguish between
binary and text, for instance in C it is the same thing, however in
Python one has type `str` whereas the other has type `bytes`.

You are not expected to write any complex algorithm here, just
delegate to the correct utility functions of your language if they are
provided. In other words don't rewrite yet another routine to
hand-parse hexadecimal.

In [6]:
def bin2txt(arg: bytes) -> str:
    return arg.decode("utf-8")

def bin2hex(arg: bytes) -> str:
    return arg.hex()

def txt2bin(arg: str) -> bytes:
    return arg.encode()

def hex2bin(arg: str) -> bytes:
    return bytes.fromhex(arg)

def hex2txt(arg: str) -> str:
    return bin2txt(hex2bin(arg))

def txt2hex(arg: str) -> str:
    return bin2hex(txt2bin(arg))

## Exercise 1: encrypt xor

Write a function that encrypts a text with a xor key. The idea is
simple: to obtain the first byte of the ciphertext, xor the first byte
of the text and the first byte of the key. Continue in this fashion
until the text is exhausted. If the key is shorter than the text, it
should be recycled (start over from the beginning).

For instance, xorring the text `everything remains raw` with the key
`word up` should give you the following hexadecimal ciphertext:
`121917165901181e01154452101d16061c1700071100`.

What is the ciphertext of the hex-encoded text `the world is yours` against the key `illmatic`?

In [40]:
from itertools import cycle

def xor(clear_text: bytes, key: bytes) -> bytes:
    # fancy way how to do key[i % len(key)]
    key_generator = cycle(key)
    # we don't like for cycles, so let's zip bytes from clear text and key
    return bytes([x^y for x,y in zip(clear_text, key_generator)])

# test part
clear_text = txt2bin('everything remains raw')
key = txt2bin('word up')
expected_cipher_text = hex2bin('121917165901181e01154452101d16061c1700071100')
cipher_text = xor(clear_text, key)

assert cipher_text == expected_cipher_text
print(f'Generated cipher text matches expected: {cipher_text == expected_cipher_text}')

# now lets get real data
clear_text = txt2bin('the world is yours')
key = txt2bin('illmatic')
cipher_text = xor(clear_text, key)
print(f'Hex Ciphertext: {bin2hex(cipher_text)}')

Generated cipher text matches expected: True
Hex Ciphertext: 1d04094d161b1b0f0d4c051e410d06161b1f


## Exercise 2: decrypt single-letter xor

The following hex-encoded ciphertext was encoded against the
single-letter key `$`, i.e. ASCII 36.

    404b48484504404b48484504464d4848045d4b

Before decrypting, shortly explain what pattern(s) are present in this
ciphertext due to the weak mode of encryption.

Then, decrypt the text. What is the plaintext?

In [41]:
cipher_text = hex2bin('404b48484504404b48484504464d4848045d4b')
print(cipher_text)

b'@KHHE\x04@KHHE\x04FMHH\x04]K'


When we print the cipher text in binary, we can see pattern `KHHE` repeating two times and `\x04` repeating three times. This suggests that there is one word two times and one symbol three times. Also, `HH` is repeating three times so there's repeating symbol as well.

In [42]:
key = txt2bin('$')

print(f'Clear text: "{bin2txt(xor(cipher_text, key))}"')

Clear text: "dolla dolla bill yo"


And yes, now we can map that `KHHE` was `dolla`, repeating twice, `\x04` was space and `HH` was `ll`.

## Exercise 3: hand crack single-letter xor

The file `text1.hex` contains a hex-encoded ciphertext that was
xor encoded with a single letter.

Decrypt it. What is the first line?

*(I think it is just easier to go right into excercise **4**)*

## Exercise 4: automate cracking single-letter xor

Solve the previous exercise, but instead of searching for the correct
key/plaintext with your eyes, make the computer do it. In other words,
you should have a function that, given a single-letter xor encoded
ciphertext, will return you the single-byte key (and, if you want, the
plaintext).

You could devise a scoring function that checks, for a given
decryption, if it seems like English. Then just iterate through all
possible keys and return the key whose decryption gets the best
score.

In [98]:
# https://inventwithpython.com/hacking/chapter20.html
FREQUENCY_OF_ENGLISH_LETTERS = {'E': 12.70, 'T': 9.06, 'A': 8.17, 'O': 7.51, 'I': 6.97, 'N': 6.75, 'S': 6.33, 'H': 6.09, 'R': 5.99, 'D': 4.25, 'L': 4.03, 'C': 2.78, 'U': 2.76, 'M': 2.41, 'W': 2.36, 'F': 2.23, 'G': 2.02, 'Y': 1.97, 'P': 1.93, 'B': 1.29, 'V': 0.98, 'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10, 'Z': 0.07}

In [142]:
import string
from collections import Counter

def contains_only_printable_chars(clear_text: str) -> bool:
    # check if there're any non printable characters
    return all([char in string.printable for char in clear_text])

def english_frequency_analysis(clear_text: str, diff = 5) -> float:
    if not contains_only_printable_chars(clear_text):
        return float("inf")

    # now we need only text that can be used for frequency analysis
    normalized = [l for l in clear_text.upper() if l in FREQUENCY_OF_ENGLISH_LETTERS.keys()]
    
    counter = Counter(normalized)
    
    # calculate relative frequency of letters
    relative_frequencies = {letter: (100*count)/len(normalized) for letter, count in counter.items()}
    # and calculate final score for the text
    return sum([abs(FREQUENCY_OF_ENGLISH_LETTERS[letter] - freq) for letter, freq in relative_frequencies.items()])

# returns key, score, plaintext
def decrypt_cipher(cipher_text: bytes, alphabet: [bytes]) -> [str, float, str]:
    score = {}
    plain_texts = {}
    for key in alphabet:
        plain_texts[key] = bin2txt(xor(cipher_text, key))
        score[key] = english_frequency_analysis(plain_texts[key])
        
    key = min(score, key=score.get)
    return bin2txt(key), score[key], plain_texts[key]

In [143]:
with open("text1.hex") as f:
    cipher_text = hex2bin(f.read())

# generate all possible keys
alphabet = [txt2bin(letter) for letter in string.ascii_letters]

key, score, plain_text = decrypt_cipher(cipher_text, alphabet)

print(f'Encryption key: "{key}" with frequency score {score}')
plain_text.split("\n")[0]

Encryption key: "M" with frequency score 26.15073394495413


'Busta Rhymes up in the place, true indeed'

## Exercise 5: crack multiple-letter xor with given key length

The file `text2.hex` contains a hex-encoded ciphertext that was xor
encoded against a multiple-letter key -- just like Ex 1.

Crack it. You are given the following indication: the key contain
10 characters.

Notice that by a simple manipulation of the ciphertext, the 10-letter
encryption is nothing more than a collection of 10 single-letter
encryptions -- which you can already crack thanks to Ex 4.

What is the key, what is the first line?

In [144]:
def decrypt_cipher_chunking(cipher_text: bytes, key_length: int, alphabet: [bytes]):
    cipher_text_chunks = [cipher_text[i::key_length] for i in range(key_length)]
    keys = []
    for chunk in cipher_text_chunks:
        key, score, plain_text = decrypt_cipher(chunk, alphabet)
        keys.append(key)
    
    key = ''.join(keys)
    return key, bin2txt(xor(cipher_text, txt2bin(key)))

In [145]:
with open("text2.hex") as f:
    cipher_text = hex2bin(f.read())

alphabet = [txt2bin(l) for l in string.printable]
key, plain_text = decrypt_cipher_chunking(cipher_text, 10, alphabet)
print(f'Key: {key}')
plain_text.split('\n')[0]

Key: SupremeNTM


"C'est le nouveau, phenomenal, freestyle du visage pale"

## Exercise 6: crack multiple-letter xor with unknown key length

Decrypt `text3.hex`, except this time you don't know the keylength.
Even better if you can make your code find out the keylength before
trying to decrypt.

What is the key, what is the first line?

In [152]:
with open("text3.hex") as f:
    cipher_text = hex2bin(f.read())

score = {}
plain_texts = {}
keys = []

for length in range(2, 258):
    print(f'Testing key length {length}...')
    key, plain_text = decrypt_cipher_chunking(cipher_text, length, alphabet)
    score[key] = english_frequency_analysis(plain_text)
    plain_texts[key] = plain_text
    keys.append(key)
    # trashold check, it is unlikely that english text would
    # have score higher then this, and even if it has
    # we select by min anyway at the end
    if score[key] < 50:
        break
    
key = min(score, key=score.get)
print(f'Key: {key}')
plain_text.split('\n')[0]

Testing key length 2...
Testing key length 3...
Testing key length 4...
Testing key length 5...
Testing key length 6...
Testing key length 7...
Testing key length 8...
Testing key length 9...
Testing key length 10...
Testing key length 11...
Testing key length 12...
Testing key length 13...
Testing key length 14...
Testing key length 15...
Testing key length 16...
Key: CL4SS!C_TIM3L35S


"And now for my next number I'd like to return to the..."

## Bonus: when you have finished all exercises

A careless user of cryptography has reused a classic timeless key to
encrypt the file `secret.zip`, which reveals the way to an important
philosophical work.

According to this masterpiece, what comes brand new?

Obviously, the password for the zip file was `CL4SS!C_TIM3L35S` from the previous excercise. The content of the `philosophy.txt` was then `PMbELEUfmIA`. Quick google reveals that this is youtube URL - https://www.youtube.com/watch?v=PMbELEUfmIA

According to the song's lyrics:
> Here comes the brand new flava in ya ear

The answer is "flava in ya ear".

## Exercise 2.1: why we do all of this

This is the easiest exercise, but also the most important one.

Write in your report the following sentence:

    I, <your name here>, understand that cryptography is easy to mess up, and
    that I will not carelessly combine pieces of cryptographic ciphers to
    encrypt my users' data. I will not write crypto code myself, but defer to
    high-level libaries written by experts who took the right decisions for me,
    like NaCL.

That's it. You will indeed get points for writing this sentence in your report.


    I, Lukáš Forst, understand that cryptography is easy to mess up, and
    that I will not carelessly combine pieces of cryptographic ciphers to
    encrypt my users' data. I will not write crypto code myself, but defer to
    high-level libaries written by experts who took the right decisions for me,
    like NaCL.